In [4]:
import pandas as pd
import numpy as np

In [5]:
from utils.EmbeddingVectorizer import tfidf_vectorizer, bow_vectorizer, embedding_vectorizer

In [6]:
train_data = pd.read_csv("../dataset/train_data.csv")
test_data = pd.read_csv("../dataset/test_data.csv")
X_train = train_data.iloc[:, 0]
X_test = test_data.iloc[:, 0]

In [7]:
Xtrain_tfidf, Xtest_tfidf = tfidf_vectorizer(X_train=X_train, X_test=X_test)
print(Xtrain_tfidf.shape)
print(Xtest_tfidf.shape)

(1600, 18450)
(400, 18450)


In [8]:
Xtrain_bow, Xtest_bow = bow_vectorizer(X_train=X_train, X_test=X_test)
print(Xtrain_bow.shape)
print(Xtest_bow.shape)

(1600, 18450)
(400, 18450)


In [9]:
Xtrain_tfidf[1][5]

0.0

In [11]:
Xtrain_em, Xtest_em = embedding_vectorizer(X_train=X_train.to_list(), X_test=X_test.to_list())
print(Xtrain_em.shape)
print(Xtest_em.shape)

EmbeddingVectorizer initialized with model: intfloat/multilingual-e5-base on device: cpu
(1600, 768)
(400, 768)


In [2]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.1f} GB")
else:
    print("No GPU available")

PyTorch version: 2.2.2+cu121
CUDA available: True
CUDA version: 12.1
Device count: 1
GPU 0: NVIDIA RTX 500 Ada Generation Laptop GPU
Memory: 3.7 GB


In [3]:
try:
    # Test 1: Tạo tensor nhỏ
    print("Test 1: Creating small tensor...")
    x = torch.tensor([1.0, 2.0, 3.0], device='cuda')
    print("✅ Success")

    # Test 2: Phép toán đơn giản
    print("Test 2: Simple operation...")
    y = x * 2
    print(f"✅ Result: {y}")

    # Test 3: Tensor lớn hơn
    print("Test 3: Larger tensor...")
    z = torch.randn(100, 100, device='cuda')
    print("✅ Success")

    # Test 4: Phép nhân ma trận
    print("Test 4: Matrix multiplication...")
    result = torch.mm(z, z.T)
    print("✅ Success")

except Exception as e:
    print(f"❌ Error at step: {e}")

Test 1: Creating small tensor...
✅ Success
Test 2: Simple operation...
✅ Result: tensor([2., 4., 6.], device='cuda:0')
Test 3: Larger tensor...
✅ Success
Test 4: Matrix multiplication...
✅ Success


In [26]:
torch.cuda.empty_cache()
if torch.cuda.is_available():
    torch.cuda.synchronize()
    print("CUDA context reset")

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def rf_model(trial):
        """
        model random forest to find the initial accuracy before optimization
        :param trial: the objective parameters for optuna
        :return: first accuracy
        """
        para_grid = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 100),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 5),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 2, 5),
            "max_features": trial.suggest_float("max_features", 0.1, 1.0),
            "class_weight": trial.suggest_categorical("class_weight", [None, "balanced"]),
            "random_state": 42,
            "n_jobs": -1,
        }
        # first train
        rfc = RandomForestClassifier(**para_grid)
        rfc.fit(X_train, y_train)
        y_pred = rfc.predict(X_test)
        score = cross_val_score(rfc, X_train, y_train, cv=5, n_jobs=-1).mean()
        return score

    study = optuna.create_study(direction="maximize")
    study.optimize(rf_model, n_trials=20)

    # Collect best parameters
    best_parameters = study.best_params
    print("Best parameters found:")
    for key, value in best_parameters.items():
        print(f"    {key}: {value}")

    # Apply to new optimized model
    optimized_rfc = RandomForestClassifier(**best_parameters)
    optimized_rfc.fit(X_train, y_train)
    y_pred = optimized_rfc.predict(X_test)

    # Calculate metrics then export report:
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return y_pred, accuracy, report